In [318]:
import pandas as pd
import numpy as np
import math
import itertools
import re


class arayi(object):
    
    def removeCols(self, csvPath):
        df=pd.read_csv(csvPath)
        dropCols = range(0,15)
        dropCols += range(16,29)
        df.drop(df.columns[dropCols],axis=1,inplace=True)
        return df

    def calcFAC(self,df):
        facDict={}
        for idx, row in df.iterrows():
            ## Build dictionary with track keys
            for cell in row[1:11]:
                if cell.split("/")[5] not in facDict:
                    facDict[cell.split("/")[5]]={"familiar":0,"listen":0,"sing":0}

            ## Count number of times track is familiar
            for loc, famil in enumerate(row[11:21]):
                if famil == "Y":
                    facDict[row[loc+1].split("/")[5]]["familiar"]+=1

            # Count number of times a song was preferred to listen
            for loc, listen in enumerate(row[28:33]):
                if listen == 1:
                    header="Input.link%sa" % str(loc+1)
                    facDict[row[header].split("/")[5]]["listen"]+=1
                else:
                    header="Input.link%sb" % str(loc+1)
                    facDict[row[header].split("/")[5]]["listen"]+=1
            
            # Count number of times a song was preferred to be sung
            for loc, sing in enumerate(row[51:56]):
                if sing == 1:
                    header="Input.link%sa" % str(loc+1)
                    facDict[row[header].split("/")[5]]["sing"]+=1
                else:
                    header="Input.link%sb" % str(loc+1)
                    facDict[row[header].split("/")[5]]["sing"]+=1

        return facDict

    def calcAHP(self,df):
        ahpDict={"globals":{"famil":{"genre":[],"repro":[],"listen":[]},"genre":{"listen":[],"repro":[]},"listen":{"repro":[]}},"locals":{"famil":{"Yes":[]},"listen":{"Low":{"Mid":[],"High":[]},"Mid":{"High":[]}},"repro":{"Low":{"Mid":[],"High":[]},"Mid":{"High":[]}},"genre":{"Rock":{"Pop":[],"Alt":[],"Country":[],"Rap":[]},"Pop":{"Alt":[],"Country":[],"Rap":[]},"Alt":{"Country":[],"Rap":[]},"Country":{"Rap":[]}}}}

        for idx, row in df.iterrows():

            ## Globals
            for ind,glob in enumerate(row[22:28]):
                header = df.columns[ind+22].split("_")[1]
                if glob == 1:
                    glob=0.0
                else:
                    glob=float(glob)

                if np.sign(glob) == 1:
                    try:
                        ahpDict["globals"][header.split("-")[1]][header.split("-")[0]].append(glob)
                    except:
                        ahpDict["globals"][header.split("-")[0]][header.split("-")[1]].append(1/glob)
                elif np.sign(glob) == -1:
                    try:
                        ahpDict["globals"][header.split("-")[0]][header.split("-")[1]].append(abs(glob))
                    except:
                        ahpDict["globals"][header.split("-")[1]][header.split("-")[0]].append(abs(1/glob))
                else:
                    try:
                        ahpDict["globals"][header.split("-")[0]][header.split("-")[1]].append(1.0)
                    except:
                        ahpDict["globals"][header.split("-")[1]][header.split("-")[0]].append(1.0)

            ## Locals
#             print df.columns[33:50]
            
            # Familiar
            famLoc = float(row[33])
            if np.sign(famLoc) < 0:
                ahpDict["locals"]["famil"]["Yes"].append(abs(famLoc))
            elif np.sign(famLoc) > 0:
                ahpDict["locals"]["famil"]["Yes"].append(1/famLoc)
            else:
                ahpDict["locals"]["famil"]["Yes"].append(1.0)
        
            # Genre
            for dex,locs in enumerate(row[34:44]):
                header=df.columns[dex+34].split("-")[1]
                header=re.findall(regexStr, header)
                locs=float(locs)
                if np.sign(locs) > 1:
                    try:
                        ahpDict["locals"]["genre"][header[1]][header[0]].append(locs)
                    except:
                        ahpDict["locals"]["genre"][header[0]][header[1]].append(1/locs)
                elif np.sign(locs) < 0:
                    try:
                        ahpDict["locals"]["genre"][header[0]][header[1]].append(abs(locs))
                    except:
                        ahpDict["locals"]["genre"][header[1]][header[0]].append(1/abs(locs))
                else:
                    try:
                        ahpDict["locals"]["genre"][header[0]][header[1]].append(1.0)
                    except:
                        ahpDict["locals"]["genre"][header[1]][header[0]].append(1.0)
                        
            # Listen
            for dex,locs in enumerate(row[44:47]):
                header=df.columns[dex+44].split("-")[1]
                header=re.findall(regexStr,header)
                locs=float(locs)
                if np.sign(locs) > 1:
                    try:
                        ahpDict["locals"]["listen"][header[1]][header[0]].append(locs)
                    except:
                        ahpDict["locals"]["listen"][header[0]][header[1]].append(1/locs)
                elif np.sign(locs) < 0:
                    try:
                        ahpDict["locals"]["listen"][header[0]][header[1]].append(abs(locs))
                    except:
                        ahpDict["locals"]["listen"][header[1]][header[0]].append(1/abs(locs))
                else:
                    try:
                        ahpDict["locals"]["listen"][header[0]][header[1]].append(1.0)
                    except:
                        ahpDict["locals"]["listen"][header[1]][header[0]].append(1.0)                    
                        
            # Repro
            for dex,locs in enumerate(row[44:47]):
                header=df.columns[dex+44].split("-")[1]
                header=re.findall(regexStr,header)
                locs=float(locs)
                if np.sign(locs) > 1:
                    try:
                        ahpDict["locals"]["repro"][header[1]][header[0]].append(float(locs))
                    except:
                        ahpDict["locals"]["repro"][header[0]][header[1]].append(1/float(locs))
                elif np.sign(locs) < 0:
                    try:
                        ahpDict["locals"]["repro"][header[0]][header[1]].append(float(locs))
                    except:
                        ahpDict["locals"]["repro"][header[1]][header[0]].append(1/float(locs))                    
                else:
                    try:
                        ahpDict["locals"]["repro"][header[0]][header[1]].append(1.0)
                    except:
                        ahpDict["locals"]["repro"][header[1]][header[0]].append(1.0)                                     
        return ahpDict
        
        
a=arayi()
regexStr='[A-Z][a-z]*'
data=a.removeCols("/Users/nus/Documents/GitHub/Singability/data/mTurkResults.csv")

facDict=a.calcFAC(data)

ahpDict=a.calcAHP(data)

# print ahpDict["globals"]
for key in ahpDict["globals"]:
    for value in ahpDict["globals"][key]:
        print key, value, np.mean(ahpDict["globals"][key][value])

print ahpDict["locals"].keys()
# print ahpDict["locals"]["genre"]["Rock"]["Rap"]
for key in ahpDict["locals"]:
    for level in ahpDict["locals"][key]:
#         print key,level
#         print key,level,ahpDict["locals"][key][level].keys()
        for value in ahpDict["locals"][key][level]:
            print "local", key,level,value, np.mean(ahpDict["locals"][key][level][value])



genre repro 1.20392871608
genre listen 0.946942081814
famil genre 1.4931146213
famil repro 1.30052652896
famil listen 1.1541109761
listen repro 1.43195625759
['genre', 'repro', 'famil', 'listen']
genre ['Country', 'Alt', 'Pop', 'Rock']
local genre Country Rap 0.830903199676
local genre Alt Country 0.849129202106
local genre Alt Rap 1.35236938032
local genre Pop Country 1.4447144593
local genre Pop Alt 0.909477521264
local genre Pop Rap 1.40583232078
local genre Rock Country 0.846294046173
local genre Rock Alt 1.28918590522
local genre Rock Rap 1.42648845687
local genre Rock Pop 1.26974483597
repro ['Low', 'Mid']
local repro Low High 0.133252328878
local repro Low Mid 0.63304981774
local repro Mid High 0.483596597813
famil ['Yes']
local famil Yes 3.0

TypeError: list indices must be integers, not float